In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from datetime import timedelta

pd.set_option('display.max_columns', None)

In [2]:
df_click_new = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/click_stream_new.csv')
df_customer = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/customer.csv')
df_product = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/product.csv', on_bad_lines='skip')
df_trans_new = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/transaction_new.csv')

In [3]:
# 2020.08.01부터 2022.07.31까지의 데이터만 필터링
df_click_new['event_time'] = pd.to_datetime(df_click_new['event_time'])
start_date = pd.to_datetime('2020-08-01', utc=True).tz_convert('UTC')
end_date = pd.to_datetime('2022-07-31', utc=True).tz_convert('UTC')
df_click_filtered = df_click_new[(df_click_new['event_time'] >= start_date) & (df_click_new['event_time'] <= end_date)]


### 중고 키워드 잡기

In [4]:
df_click_new['search_keywords'].value_counts()

search_keywords
Dress Kondangan    163516
Tas Wanita          82082
Bekas               71962
T-Shirt             64792
Baju                53912
Kaos                53767
Dress Pesta         49025
Sepatu Nike         33033
Tas Kerja           24751
Second Hand         18208
Second              18099
Thrift              18098
Celana Panjang      17980
Sepatu              17953
Barang Bekas        17838
Sepatu Adidas       16487
Sendal               8211
Celana               7770
T-shirt              4715
Jam                  4704
Topi                 3215
Baju Tidur           1543
Daster                803
Name: count, dtype: int64

In [5]:
# 한글 단어로 변경할 매핑 딕셔너리 생성
mapping = {
    'Dress Kondangan': '파티 드레스',
    'Tas Wanita': '여성 가방',
    'Bekas': '중고',
    'T-Shirt': '티셔츠',
    'Baju': '옷',
    'Kaos': '셔츠',
    'Dress Pesta': '파티 드레스',
    'Sepatu Nike': '나이키 신발',
    'Tas Kerja': '직장용 가방',
    'Second Hand': '중고',
    'Second': '중고',
    'Thrift': '중고',
    'Sepatu': '신발',
    'Celana Panjang': '긴 바지',
    'Barang Bekas': '중고',
    'Sepatu Adidas': '아디다스 신발',
    'Sendal': '샌들',
    'Celana': '바지',
    'T-shirt': '티셔츠',
    'Jam': '시계',
    'Topi': '모자',
    'Baju Tidur': '잠옷',
    'Daster': '원피스'
}

# 'search_keywords' 열의 값을 매핑 딕셔너리를 사용하여 변경
df_click_new['search_keywords'] = df_click_new['search_keywords'].map(mapping)

In [6]:
df_click_new['search_keywords'].value_counts()

search_keywords
파티 드레스     212541
중고         144205
여성 가방       82082
티셔츠         69507
옷           53912
셔츠          53767
나이키 신발      33033
직장용 가방      24751
긴 바지        17980
신발          17953
아디다스 신발     16487
샌들           8211
바지           7770
시계           4704
모자           3215
잠옷           1543
원피스           803
Name: count, dtype: int64

In [7]:
df_click_new.head(5)

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
741,50761ff1-f1e9-4680-a626-c4282d70dd6f,HOMEPAGE,2020-08-02 11:58:25.425431+00:00,a97da81c-6c2d-43b6-aac4-816c66c27691,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,50761ff1-f1e9-4680-a626-c4282d70dd6f,ADD_TO_CART,2020-08-02 12:42:22.425431+00:00,0b870966-66c0-4b60-9093-0b2887d301f2,MOBILE,35575.0,1.0,255203.0,NaN,NaN,NaN,NaN
743,50761ff1-f1e9-4680-a626-c4282d70dd6f,ADD_PROMO,2020-08-02 13:25:34.425431+00:00,49b0bd63-1692-4dbb-87cf-2d5b05e9221e,MOBILE,NaN,NaN,NaN,NaN,NaN,WEEKENDSERU,2003.0
744,50761ff1-f1e9-4680-a626-c4282d70dd6f,BOOKING,2020-08-02 19:19:28.425431+00:00,6ad88732-bf72-486e-bab4-d3ca592d0b32,MOBILE,NaN,NaN,NaN,Failed,NaN,NaN,NaN
745,50761ff1-f1e9-4680-a626-c4282d70dd6f,PROMO_PAGE,2020-08-02 12:42:44.425431+00:00,536122d8-6625-462c-a056-b28ba4ced34b,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("세션에서 검색했을 때 그것이 구매시도로 이어지는 확률 :", average_probability)

NameError: name 'average_probability' is not defined

In [ ]:
# 각 세션의 시작 시간과 종료 시간 계산
session_start_time = df_click_new.groupby('session_id')['event_time'].min()
session_end_time = df_click_new.groupby('session_id')['event_time'].max()

# 세션의 지속 시간 계산 및 exist_time 열 생성
df_click_new['duration_time'] = df_click_new['session_id'].map(session_end_time - session_start_time)

# 평균 체류 시간 계산
average_stay_time = df_click_new['duration_time'].mean()

print("평균 체류 시간 :", average_stay_time)

평균 체류 시간 : 10 days 00:55:07.797743152


In [ ]:
df_click_new.describe()

,product_id,quantity,item_price,promo_amount,duration_time
count,1.136832e+06,1.136832e+06,1.136832e+06,183973.000000,7995246
mean,2.917746e+04,1.441392e+00,2.661372e+05,5336.722389,10 days 00:55:07.797743152
std,1.703540e+04,1.494611e+00,1.177385e+05,2644.506784,19 days 13:34:43.200125632
min,1.163000e+03,1.000000e+00,1.032400e+04,235.000000,0 days 00:00:00
25%,1.405100e+04,1.000000e+00,1.802818e+05,3446.000000,1 days 03:28:24
50%,2.804000e+04,1.000000e+00,2.493000e+05,4844.000000,4 days 02:29:09
75%,4.393200e+04,1.000000e+00,3.336260e+05,6662.000000,10 days 18:52:59
max,6.000000e+04,4.300000e+01,1.335959e+06,24519.000000,462 days 02:16:55


In [ ]:
df_click_new[df_click_new['session_id'] == '93325fb6-eb00-4268-bb0e-6471795a0ad0']

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,duration_time


In [ ]:
ratio_book =  df_click_new[df_click_new['event_name'] == 'BOOKING']['session_id'].count()/df_click_new['session_id'].nunique()
ratio_book

0.9651796793950549

In [ ]:
# 'session_id'로 그룹화하여 각 그룹에서 'booking' 이벤트가 없는 세션 필터링
df_click_new.groupby('session_id').filter(lambda x: 'BOOKING' not in x['event_name'].values)

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,duration_time
51627,a77afa56-7a3f-4015-99f3-58316d07e186,HOMEPAGE,2022-07-30 05:21:36.270467+00:00,94d3c5a2-0f9d-424c-99e7-204c4ed71e01,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51628,a77afa56-7a3f-4015-99f3-58316d07e186,ADD_TO_CART,2022-07-30 07:40:52.270467+00:00,9e50f854-d012-4ce1-ab92-7fb9f6a72872,MOBILE,18827.0,5.0,374599.0,NaN,NaN,NaN,NaN,0 days 18:23:48
51629,a77afa56-7a3f-4015-99f3-58316d07e186,ADD_PROMO,2022-07-30 09:58:33.270467+00:00,da541f1e-60fa-4254-bff6-364e48ce24f5,MOBILE,NaN,NaN,NaN,NaN,NaN,WEEKENDSERU,2986.0,0 days 18:23:48
51631,a77afa56-7a3f-4015-99f3-58316d07e186,CLICK,2022-07-30 07:39:35.270467+00:00,bb9bb8ca-96af-4ab7-88d4-0197806079be,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51632,a77afa56-7a3f-4015-99f3-58316d07e186,SCROLL,2022-07-30 09:55:48.270467+00:00,773b34d5-5a64-4cf3-ba78-4a179578ab27,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833211,5a177399-8fda-481e-91e5-cba24a6e63d8,HOMEPAGE,2021-12-15 12:43:49.276026+00:00,c1d7fc77-ee97-4256-94f4-a8950011ce91,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:10:30
12833260,4a3a58c6-4ff5-417d-9d80-e1a9a2fd7cf9,HOMEPAGE,2021-09-04 00:14:53.693223+00:00,908c0400-7743-416d-8e2c-474e3b6676a0,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:09:21
12833261,4a3a58c6-4ff5-417d-9d80-e1a9a2fd7cf9,SCROLL,2021-09-04 00:24:14.693223+00:00,b7e31929-38ef-44fc-a719-2f76857bada0,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:09:21
12833290,2f51f07f-2b5e-49da-bbd6-5ca6c193418b,HOMEPAGE,2021-04-29 12:30:07.155099+00:00,16a6613b-cda9-413a-b62c-91ce9d471d50,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:05:51


In [ ]:
df_click_new[df_click_new['duration_time'] == '0 days 18:23:48'].sort_values(['event_time'])

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,duration_time
51627,a77afa56-7a3f-4015-99f3-58316d07e186,HOMEPAGE,2022-07-30 05:21:36.270467+00:00,94d3c5a2-0f9d-424c-99e7-204c4ed71e01,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51631,a77afa56-7a3f-4015-99f3-58316d07e186,CLICK,2022-07-30 07:39:35.270467+00:00,bb9bb8ca-96af-4ab7-88d4-0197806079be,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51628,a77afa56-7a3f-4015-99f3-58316d07e186,ADD_TO_CART,2022-07-30 07:40:52.270467+00:00,9e50f854-d012-4ce1-ab92-7fb9f6a72872,MOBILE,18827.0,5.0,374599.0,NaN,NaN,NaN,NaN,0 days 18:23:48
51632,a77afa56-7a3f-4015-99f3-58316d07e186,SCROLL,2022-07-30 09:55:48.270467+00:00,773b34d5-5a64-4cf3-ba78-4a179578ab27,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51629,a77afa56-7a3f-4015-99f3-58316d07e186,ADD_PROMO,2022-07-30 09:58:33.270467+00:00,da541f1e-60fa-4254-bff6-364e48ce24f5,MOBILE,NaN,NaN,NaN,NaN,NaN,WEEKENDSERU,2986.0,0 days 18:23:48
51633,a77afa56-7a3f-4015-99f3-58316d07e186,SCROLL,2022-07-30 12:13:07.270467+00:00,8571545c-8d87-41fd-8cc3-38b50524bf35,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51634,a77afa56-7a3f-4015-99f3-58316d07e186,SEARCH,2022-07-30 14:32:11.270467+00:00,4bb2f4b5-0f29-4341-a1ec-58193506370c,MOBILE,NaN,NaN,NaN,NaN,셔츠,NaN,NaN,0 days 18:23:48
51635,a77afa56-7a3f-4015-99f3-58316d07e186,SCROLL,2022-07-30 16:50:53.270467+00:00,71c1599c-f56f-4c67-a240-2b9982dd4571,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51636,a77afa56-7a3f-4015-99f3-58316d07e186,HOMEPAGE,2022-07-30 19:07:13.270467+00:00,a84ff8b7-ccf0-4d3d-b9bb-0cd708246064,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48
51637,a77afa56-7a3f-4015-99f3-58316d07e186,HOMEPAGE,2022-07-30 21:26:18.270467+00:00,dd380132-ae12-41a9-944c-22bc5d97370f,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 18:23:48


In [ ]:
tmp = df_click_new.groupby('session_id')['event_time']
len(tmp)

550397

In [ ]:
df_click_new = df_click_new.sort_values

In [ ]:
tmp.head(5)

0         2020-08-02 11:58:25.425431+00:00
1         2020-08-02 12:42:22.425431+00:00
2         2020-08-02 13:25:34.425431+00:00
3         2020-08-02 19:19:28.425431+00:00
4         2020-08-02 12:42:44.425431+00:00
                        ...               
7995241   2021-06-07 04:37:53.354381+00:00
7995242   2021-05-05 11:44:14.985835+00:00
7995243   2021-05-05 11:53:10.985835+00:00
7995244   2021-05-05 12:10:09.135142+00:00
7995245   2021-05-05 11:53:12.985835+00:00
Name: event_time, Length: 2670710, dtype: datetime64[ns, UTC]

In [ ]:
# def redefine_session_ids(df):
#     session_id = 1
#     new_data = []

#     df = df.sort_values(['event_time'])

#     for i in range(len(df)-1):
#         current_event_time = df.iloc[i]['event_time']
        
#         if df.iloc[i]['session_id'] == df.iloc[i+1]['session_id']:
#             next_event_time = df.iloc[i + 1]['event_time']
#             time_difference = next_event_time - current_event_time

#             if time_difference > timedelta(minutes=30):
#                 session_id += 1

#         new_data.append({'session_id': session_id, 'event_time': current_event_time})

#     new_data.append({'session_id': session_id, 'event_time': df.iloc[-1]['event_time']})
#     new_df = pd.DataFrame(new_data)
#     return new_df

# new_df = redefine_session_ids(df_click_new)

In [ ]:
# def redefine_session_ids(df):
#     session_id = 1
#     new_data = []


#     for i in range(len(df)-1):
#         current_event_time = df.iloc[i]['event_time']
        
#         if df.iloc[i]['session_id'] == df.iloc[i+1]['session_id']:
#             next_event_time = df.iloc[i + 1]['event_time']
#             time_difference = next_event_time - current_event_time

#             if time_difference > timedelta(minutes=30):
#                 session_id += 1

#         new_data.append({'session_id': session_id, 'event_time': current_event_time})

#     new_data.append({'session_id': session_id, 'event_time': df.iloc[-1]['event_time']})
#     new_df = pd.DataFrame(new_data)
#     return new_df

# new_df = redefine_session_ids(df_click_new)

In [ ]:
# 세션 id 특징 파악
# 1. 하나의 세션에는 최대 1개의 BOOKING을 갖는다
# 2. BOOKING이 존재하지 않는 3~4%정도의 session_id가 존재한다
# 3. duration_time(체류시간) 계산을 통해 10일 이상의 값을 평균적으로 갖는 것을 확인
#   이는 이탈률 관련 지표 등을 계산할 때에 문제임.

In [4]:
# session_id와 event_time을 기준으로 정렬
df = df_click_new.sort_values(['session_id', 'event_time'])

# 이전 row의 event_time을 저장할 변수 초기화
prev_event_time = None

# session_id에 번호를 붙일 변수 초기화
session_num = 1

# session_id가 같은 경우, event_time 차이를 계산하고 번호를 붙임
for i, row in df.iterrows():
    if prev_event_time is not None and row['session_id'] == prev_session_id:
        time_diff = row['event_time'] - prev_event_time
        if time_diff >= timedelta(minutes=30):
            session_num += 1
    df.at[i, 'session_id'] = f"{row['session_id']}_{session_num}"
    prev_event_time = row['event_time']
    prev_session_id = row['session_id']

# 결과 출력
print(df)

                                           session_id   event_name   
469318         00001f29-3ede-4445-95f0-0178934e32ba_1     HOMEPAGE  \
469319         00001f29-3ede-4445-95f0-0178934e32ba_2  ADD_TO_CART   
469321         00001f29-3ede-4445-95f0-0178934e32ba_2     HOMEPAGE   
469322         00001f29-3ede-4445-95f0-0178934e32ba_3        CLICK   
469323         00001f29-3ede-4445-95f0-0178934e32ba_4        CLICK   
...                                               ...          ...   
5928943  fffffaf6-55ee-428b-a3bf-075cd7f1a093_8470219  ITEM_DETAIL   
5928944  fffffaf6-55ee-428b-a3bf-075cd7f1a093_8470220  ITEM_DETAIL   
5928945  fffffaf6-55ee-428b-a3bf-075cd7f1a093_8470221  ITEM_DETAIL   
5928946  fffffaf6-55ee-428b-a3bf-075cd7f1a093_8470222   PROMO_PAGE   
5928947  fffffaf6-55ee-428b-a3bf-075cd7f1a093_8470223        CLICK   

                              event_time   
469318  2019-05-17 11:51:52.276396+00:00  \
469319  2019-05-17 19:54:53.276396+00:00   
469321  2019-05-17 19:55:12

In [7]:
# 각 세션의 시작 시간과 종료 시간 계산
session_start_time = df.groupby('session_id')['event_time'].min()
session_end_time = df.groupby('session_id')['event_time'].max()

# 세션의 지속 시간 계산 및 exist_time 열 생성
df['duration_time'] = df['session_id'].map(session_end_time - session_start_time)

# 평균 체류 시간 계산
average_stay_time = df['duration_time'].mean()

print("평균 체류 시간 :", average_stay_time)

평균 체류 시간 : 0 days 00:49:11.762441822


In [8]:
before_ids = df_click_new['session_id'].nunique()
unique_session_ids = df['session_id'].nunique()

print("기존 session_id의 개수:", before_ids)
print("새로 정의한 session_id의 개수:", unique_session_ids)

기존 session_id의 개수: 895203
새로 정의한 session_id의 개수: 9365425


In [9]:
df.to_csv('model_new_session_id.csv',index = False)

In [ ]:
df_click_new = df_click_new.reset_index()

In [ ]:
df_click_new

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,duration_time,new_session
0,741,50761ff1-f1e9-4680-a626-c4282d70dd6f,HOMEPAGE,2020-08-02 11:58:25.425431+00:00,a97da81c-6c2d-43b6-aac4-816c66c27691,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 18:37:23,624.0
1,742,50761ff1-f1e9-4680-a626-c4282d70dd6f,ADD_TO_CART,2020-08-02 12:42:22.425431+00:00,0b870966-66c0-4b60-9093-0b2887d301f2,MOBILE,35575.0,1.0,255203.0,NaN,NaN,NaN,NaN,4 days 18:37:23,625.0
2,743,50761ff1-f1e9-4680-a626-c4282d70dd6f,ADD_PROMO,2020-08-02 13:25:34.425431+00:00,49b0bd63-1692-4dbb-87cf-2d5b05e9221e,MOBILE,NaN,NaN,NaN,NaN,NaN,WEEKENDSERU,2003.0,4 days 18:37:23,626.0
3,744,50761ff1-f1e9-4680-a626-c4282d70dd6f,BOOKING,2020-08-02 19:19:28.425431+00:00,6ad88732-bf72-486e-bab4-d3ca592d0b32,MOBILE,NaN,NaN,NaN,Failed,NaN,NaN,NaN,4 days 18:37:23,627.0
4,745,50761ff1-f1e9-4680-a626-c4282d70dd6f,PROMO_PAGE,2020-08-02 12:42:44.425431+00:00,536122d8-6625-462c-a056-b28ba4ced34b,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 18:37:23,628.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995241,12833597,ad1063b0-a6e5-40a9-aed6-d1b45741e571,SEARCH,2021-06-07 04:37:53.354381+00:00,4b3b91a5-9bda-44de-87dc-153367071dfc,MOBILE,NaN,NaN,NaN,NaN,중고,NaN,NaN,0 days 00:19:20.741864,NaN
7995242,12833598,64585e37-3fe7-48d7-b79b-ebcb70230eb2,HOMEPAGE,2021-05-05 11:44:14.985835+00:00,ab0acc87-9578-408a-b47f-c46996b8b52f,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:25:54.149307,NaN
7995243,12833599,64585e37-3fe7-48d7-b79b-ebcb70230eb2,ADD_TO_CART,2021-05-05 11:53:10.985835+00:00,230d7b67-9934-42f9-9509-b330ac6ae777,MOBILE,19820.0,1.0,88918.0,NaN,NaN,NaN,NaN,0 days 00:25:54.149307,NaN
7995244,12833600,64585e37-3fe7-48d7-b79b-ebcb70230eb2,BOOKING,2021-05-05 12:10:09.135142+00:00,8996ce49-c5dd-41ed-9231-28bfdb634cbe,MOBILE,NaN,NaN,NaN,Success,NaN,NaN,NaN,0 days 00:25:54.149307,NaN


In [ ]:
df_click_new.tail(50)

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,duration_time,new_session
7995196,3d8a20f8-a889-47bc-9f3a-b26534ec10e4,BOOKING,2022-04-16 15:39:37.110585+00:00,c9e8892f-5c05-4578-aaa2-eae4f5c9ec10,WEB,NaN,NaN,NaN,Success,NaN,NaN,NaN,235 days 01:23:25.541248,6147133
7995197,3d8a20f8-a889-47bc-9f3a-b26534ec10e4,CLICK,2022-04-16 15:01:38.467662+00:00,662a2e30-727f-4974-9604-64e75580eefe,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,235 days 01:23:25.541248,6147133
7995198,a8d835ef-5b34-423b-820a-892fae413831,HOMEPAGE,2021-08-24 14:16:11.569337+00:00,80263861-eaac-496b-9ae0-21b3f667d4e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,235 days 01:23:25.541248,6147133
7995199,a8d835ef-5b34-423b-820a-892fae413831,ADD_TO_CART,2021-08-24 14:16:27.569337+00:00,223e9414-7962-421e-9e04-f5f29cb2e457,MOBILE,54804.0,1.0,198447.0,NaN,NaN,NaN,NaN,235 days 01:23:25.541248,6147133
7995200,a8d835ef-5b34-423b-820a-892fae413831,BOOKING,2021-08-24 14:17:08.207000+00:00,47f8811e-c75f-4f4c-acbd-23918e093f04,MOBILE,NaN,NaN,NaN,Success,NaN,NaN,NaN,235 days 01:23:25.541248,6147133
7995201,a8d835ef-5b34-423b-820a-892fae413831,SEARCH,2021-08-24 14:16:34.569337+00:00,0eca1fd5-e573-4f82-9893-219a49b0dec3,MOBILE,NaN,NaN,NaN,NaN,여성 가방,NaN,NaN,214 days 05:43:12.967656,6147134
7995202,1ded1121-f3d0-472a-bbbf-6cc1a74c6aa2,HOMEPAGE,2022-03-26 19:59:47.536993+00:00,b98f2759-4e08-4b90-8b92-4fefb290fe7f,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214 days 05:43:12.967656,6147134
7995203,1ded1121-f3d0-472a-bbbf-6cc1a74c6aa2,ADD_TO_CART,2022-03-26 20:15:19.536993+00:00,a638c924-e8b7-4210-a87b-739d93906dee,WEB,28365.0,7.0,232356.0,NaN,NaN,NaN,NaN,83 days 03:25:36.375278,6147135
7995204,1ded1121-f3d0-472a-bbbf-6cc1a74c6aa2,BOOKING,2022-03-26 20:47:53.730622+00:00,4a5961eb-82a4-41cf-be79-633bb743cfda,WEB,NaN,NaN,NaN,Success,NaN,NaN,NaN,83 days 03:25:36.375278,6147135
7995205,1ded1121-f3d0-472a-bbbf-6cc1a74c6aa2,CLICK,2022-03-26 20:15:47.536993+00:00,9f1cb764-60d0-4255-8f26-2659cd003096,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83 days 03:25:36.375278,6147135


In [ ]:
# SEARCH 항목이 있는 각 세션의 BOOKING 포함 확률 계산
sessions_with_search = df_click_new[df_click_new['event_name'] == 'SEARCH']['session_id'].unique()  # SEARCH 항목이 있는 세션들의 고유한 세션 ID 추출
probabilities = []  # 각 세션에 대한 확률을 저장할 리스트 생성

for session in sessions_with_search:
    session_data = df_click_new[df_click_new['session_id'] == session]  # 현재 세션에 해당하는 데이터 추출
    event_names = session_data['event_name'].unique()  # 현재 세션의 모든 event_name 추출
    
    if 'BOOKING' in event_names:
        booking_probability = session_data[session_data['event_name'] == 'BOOKING'].shape[0] / session_data.shape[0]
        probabilities.append(booking_probability)

average_probability = sum(probabilities) / len(probabilities)  # 평균 확률 계산

KeyboardInterrupt: 